In [1]:
using DifferentialEquations, LinearAlgebra, Plots

test = false

function solve_ode(m,c,k)
    function F(u,p,t)
        m, c, k = p
        A = [0.0 1.0; -k/m -c/m]
        b = [0.0, 0.0]
        return A*u + b
    end
    u0 = [1.0, 0.0]
    tspan = (0.0, 1.0)
    prob = ODEProblem(F, u0, tspan, [m, c, k])
    sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)
end

if test
    sol = solve_ode(1.0, 5.0, 100.0) 
    plot(sol.t, [first(u) for u in sol.u])
end

# Hosting

In [2]:
using Diana
using Merly

In [3]:
schema = """
  type Query {
    solution: Float
  }

  schema {
    query: Query
  }
""";

In [4]:
function get_solution(root, args, ctx, info)
    m = get(args, "m", 1.0)
    c = get(args, "c", 5.0)
    k = get(args, "k", 100.0)
    solution = solve_ode(m, c, k)
    to_dict(((ui,vi),ti)) = Dict("u" => ui, "v" => vi, "t" => ti)
    return map(to_dict, tuples(sol))
end

get_solution (generic function with 1 method)

In [5]:
test && get_solution(nothing, Dict(), nothing, Dict())[1:10]

false

In [6]:
resolvers = Dict(
    "Query" => Dict(
        "solution" => get_solution
    ),
);

In [7]:
my_schema = Schema(schema, resolvers)

Diana.schema(Diana.var"#execute#43"{Diana.var"#execute#42#44"{Nothing,Dict{String,Dict{String,typeof(get_solution)}},Dict{Any,Any}}}(Diana.var"#execute#42#44"{Nothing,Dict{String,Dict{String,typeof(get_solution)}},Dict{Any,Any}}(nothing, Dict("Query" => Dict("solution" => get_solution)), Dict{Any,Any}("Query" => Dict{Any,Any}("solution" => Dict{Any,Any}("tipo" => "Float")),"query" => "Query"))))

In [8]:
test && println(my_schema.execute("{ solution(m:1.0, c:2.0, k:3.0) }"))

false

In [ ]:
Post("/graphql", (req, res) -> begin
  res.headers["Content-Type"]= "application/json"
  my_schema.execute(req.body["query"])
end)

server = Merly.app()
server.useCORS(true)
server.start(config=Dict("host" => "127.0.0.1", "port" => 4000), verbose=false)

┌ Info: Url added
│   url = POST/graphql
└ @ Merly /home/jukka/.julia/packages/Merly/f4RAG/src/routes.jl:39
